## Modules

In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
 
from datetime import datetime, timedelta
from time import time
from itertools import product

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import yfinance as yf
from tdcalendar import *

## Historical return data collection

### Price data collection

In [2]:
def getYahooData(symbolList, adjust=True, startStr='1990-01-01', endStr='2046-12-31'):
    """Scrape via yahoo API to obtain data for a symbolList."""
    symbolStr = ' '.join(symbolList)
    renameDict = {'Date': 'date', 'Open': 'op', 'High': 'hi', 'Low': 'lo', 'Close': 'cl',
                  'Volume': 'vol', 'Adj Close': 'adj_cl', 'Dividends': 'div', 'Stock Splits': 'split'}
    priceFields = ['op', 'hi', 'lo', 'cl']
    dataFields = priceFields + ['vol', 'div']

    dataDict = {}

    try:
        dfData = yf.download(symbolStr, start=startStr, end=endStr, auto_adjust=False, actions=True,
                             group_by='Tickers', threads=16)
    except:
        dfData = pd.DataFrame()

    for symbol in symbolList:
        try:
            dfSymbol = dfData[(symbol,)].dropna()  # Raw data for the symbol
            dfSymbol = dfSymbol[(dfSymbol['Volume'] > 0) | (dfSymbol['High'] > dfSymbol['Low'])]  # Filter bad data
            dfSymbol.reset_index(inplace=True)
            dfSymbol.rename(columns=renameDict, inplace=True)
            dfSymbol.set_index('date', inplace=True)

            if not adjust:
                dfSymbol = dfSymbol[dataFields]
                dfSymbol.rename(columns={field: f'{symbol}_{field}' for field in dataFields}, inplace=True)
            else:
                adjfactor = dfSymbol['adj_cl'] / dfSymbol['cl']
                for field in priceFields:
                    dfSymbol[f'adj_{field}'] = dfSymbol[field] * adjfactor
                dfSymbol['adj_vol'] = dfSymbol['vol'] / adjfactor
                dfSymbol = dfSymbol[[f'adj_{field}' for field in dataFields[:-1]]]
                dfSymbol.rename(columns={f'adj_{field}': f'{symbol}_{field}' for field in dataFields[:-1]}, inplace=True)
                dfSymbol = np.round(dfSymbol, 4)

            dataDict[symbol] = dfSymbol
        except:
            print(f'Failed preparing data for {symbol}.')

    dfAll = pd.concat(dataDict.values(), axis=1, join='inner')
    dfAll.fillna(method='ffill', inplace=True)

    return dfAll

In [3]:
symbolList = ['SPY', 'QQQ', 'IWM', 'SOXX', 'AAPL', 'MSFT', 'NVDA', 'TSLA', 'BRK-B', 'BA']

startStr = '2011-01-01'
endStr = '2023-01-01'
dfAll = getYahooData(symbolList, True, startStr, endStr)

[*********************100%***********************]  10 of 10 completed


D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2923: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)
D:\Anaconda\lib\site-p

### Dictionary of dataframes of open, close & log-change columns in training period

In [4]:
startStr_train1 = '2019-07-01'
endStr_train1 = '2022-07-01'
startDate_train1 = datetime.strptime(startStr_train1, '%Y-%m-%d')
endDate_train1 = datetime.strptime(endStr_train1, '%Y-%m-%d')

startStr_test1 = getLatestTradingDay(holidayList_ny, endDate_train1, days=0, offset=0)
endStr_test1 = '2022-12-31'
startDate_test1 = datetime.strptime(startStr_test1, '%Y-%m-%d')
endDate_test1 = datetime.strptime(endStr_test1, '%Y-%m-%d')

dataDict_train1 = {}
dataDict_test1 = {}

for symbol in symbolList:
    dfSymbol = dfAll[[f'{symbol}_op', f'{symbol}_cl']]
    dfSymbol[f'{symbol}_pct'] = np.log(dfSymbol[f'{symbol}_cl'] / dfSymbol[f'{symbol}_cl'].shift(1))
    dataDict_train1[symbol] = dfSymbol.loc[startStr_train1:endStr_train1]
    dataDict_test1[symbol] = dfSymbol.loc[startStr_test1:endStr_test1]
    
print(dataDict_train1['NVDA'].tail(10), '\n')
print(dataDict_test1['NVDA'].tail(10))

<ipython-input-4-bdaf831f32fd>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfSymbol[f'{symbol}_pct'] = np.log(dfSymbol[f'{symbol}_cl'] / dfSymbol[f'{symbol}_cl'].shift(1))


             NVDA_op   NVDA_cl  NVDA_pct
date                                    
2022-06-17  156.3669  158.6853  0.017726
2022-06-21  164.6310  165.5403  0.042292
2022-06-22  162.1428  163.4818 -0.012513
2022-06-23  165.0707  162.1328 -0.008286
2022-06-24  164.8808  171.1363  0.054045
2022-06-27  172.9949  168.5681 -0.015120
2022-06-28  168.8779  159.7045 -0.054015
2022-06-29  158.0257  155.3077 -0.027917
2022-06-30  153.4990  151.4805 -0.024951
2022-07-01  148.8824  145.1251 -0.042861 

             NVDA_op   NVDA_cl  NVDA_pct
date                                    
2022-12-16  168.6114  165.6819 -0.022731
2022-12-19  165.6919  162.5124 -0.019315
2022-12-20  160.6127  160.8227 -0.010452
2022-12-21  161.1126  164.9820  0.025534
2022-12-22  160.9127  153.3640 -0.073022
2022-12-23  151.9342  152.0342 -0.008709
2022-12-27  150.7144  141.1860 -0.074027
2022-12-28  139.2464  140.3362 -0.006037
2022-12-29  143.9956  146.0052  0.039601
2022-12-30  143.3157  146.1152  0.000753


### Descriptive Statistics of historical daily returns

In [5]:
dfStat_train = pd.DataFrame()

for symbol, df in dataDict_train1.items():
    dfStat_train[symbol] = df.describe()[f'{symbol}_pct']
    
dfStat_train

,SPY,QQQ,IWM,SOXX,AAPL,MSFT,NVDA,TSLA,BRK-B,BA
count,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000
mean,0.000412,0.000568,0.000175,0.000739,0.001394,0.000912,0.001673,0.003595,0.000348,-0.001238
std,0.014985,0.017578,0.018954,0.023961,0.022159,0.020432,0.033142,0.044341,0.015375,0.037540
min,-0.115886,-0.127592,-0.142335,-0.165202,-0.137708,-0.159453,-0.203979,-0.236518,-0.100838,-0.272444
25%,-0.004946,-0.006912,-0.008644,-0.010800,-0.008538,-0.008609,-0.015402,-0.017704,-0.006363,-0.016361
50%,0.001302,0.001803,0.000999,0.001203,0.001479,0.001219,0.002800,0.002982,0.000624,-0.002076
75%,0.007545,0.009538,0.010466,0.014640,0.013506,0.011553,0.019618,0.024721,0.007286,0.013798
max,0.086731,0.081308,0.087545,0.102701,0.113158,0.132929,0.158340,0.181446,0.109840,0.217678


### Dummy dataframe of zero-return forward price time-series

In [6]:
tradeStrList_test1 = getTradingDays(holidayList_ny, startDate_test1, endDate_test1)
tradeDateList_test1 = [datetime.strptime(dtStr, '%Y-%m-%d') for dtStr in tradeStrList_test1]

fwdDict = {}

for symbol, df in dataDict_train1.items():
    dfFwd = pd.DataFrame()
    latestPrice = df.loc[startStr_test1, f'{symbol}_cl']
    for date in tradeDateList_test1:
        dfFwd.loc[date, f'pct-0'] = 0
        dfFwd.loc[date, f'cl-0'] = latestPrice
    fwdDict[symbol] = dfFwd
        
fwdDict['NVDA'].head(10)

,pct-0,cl-0
2022-07-01,0.0,145.1251
2022-07-05,0.0,145.1251
2022-07-06,0.0,145.1251
2022-07-07,0.0,145.1251
2022-07-08,0.0,145.1251
2022-07-11,0.0,145.1251
2022-07-12,0.0,145.1251
2022-07-13,0.0,145.1251
2022-07-14,0.0,145.1251
2022-07-15,0.0,145.1251


## Monte-Carlo Resampling (MCR)

### Construction of 1x leverage forward time series by MCR

In [7]:
def getSim_mcr1x(dfData, dateList, startPrice=1.0, numSim=400):
    """Obtain the dataframe of MCR simulations of forward price time-series."""
    # Build MCR dataframe by dummy zero-return price time-series
    dfSim = pd.DataFrame()
    for date in dateList:
        dfSim.loc[date, f'pct-0'] = 0
        dfSim.loc[date, f'cl-0'] = latestPrice
    # Historical return array     
    pctArray = np.array(dfData[f'{symbol}_pct'])
    numData = len(pctArray)
    # MCR simulations
    lenPeriod = len(dateList) - 1

    for num in range(numSim):
        ordinalArray = np.random.randint(numData, size=lenPeriod)
        fwdPctArray = [0] + [pctArray[n] for n in ordinalArray]
        dfSim[f'pct-{num + 1}'] = fwdPctArray
        dfSim[f'cl-{num + 1}'] = startPrice * np.exp(dfSim[f'pct-{num + 1}'].cumsum())
    
    return dfSim        

def fitActualvsMCR(dfSim, dfData, dateList):
    """Fit the actual price time-series into an MCR dataframe."""
    dfSim['cl-0'] = dfData.loc[dateList, f'{symbol}_cl']
    dfSim['pct-0'] = dfData.loc[dateList, f'{symbol}_pct']
    
    return dfSim

In [8]:
for symbol in symbolList:
    dfData_train = dataDict_train1[symbol]
    dfData_test = dataDict_test1[symbol]
    fwdDict[symbol] = getSim_mcr1x(dfData_train, tradeDateList_test1, dfData_train[f'{symbol}_cl'].iloc[-1]) 
    fwdDict[symbol] = fitActualvsMCR(fwdDict[symbol], dfData_test, tradeDateList_test1)
    print(f'{symbol} MCR done.')

SPY MCR done.
QQQ MCR done.
IWM MCR done.
SOXX MCR done.
AAPL MCR done.
MSFT MCR done.
NVDA MCR done.
TSLA MCR done.
BRK-B MCR done.
BA MCR done.


In [9]:
fwdDict['NVDA'].iloc[-10:, :12]

,pct-0,cl-0,pct-1,cl-1,pct-2,cl-2,pct-3,cl-3,pct-4,cl-4,pct-5,cl-5
2022-12-16,-0.022731,165.6819,0.014739,192.546731,0.023854,169.423661,0.040539,106.329103,-0.054015,149.725046,-0.057586,157.740480
2022-12-19,-0.019315,162.5124,-0.017035,189.294431,0.060508,179.991664,-0.063050,99.831994,0.007429,150.841498,0.003200,158.246066
2022-12-20,-0.010452,160.8227,-0.006550,188.058608,0.031098,185.677074,-0.017491,98.100985,-0.019049,147.995254,-0.010643,156.570792
2022-12-21,0.025534,164.9820,-0.005375,187.050417,-0.001950,185.315285,-0.042885,93.982862,0.019256,150.872665,-0.015098,154.224667
2022-12-22,-0.073022,153.3640,0.027562,192.277624,-0.042885,177.536045,-0.023291,91.819192,-0.010729,149.262650,0.031836,159.213616
2022-12-23,-0.008709,152.0342,-0.010278,190.311588,-0.009128,175.922920,0.067128,98.194451,-0.012513,147.406563,0.010887,160.956477
2022-12-27,-0.074027,141.1860,0.023126,194.763993,-0.052198,166.975713,0.027994,100.982194,0.004762,148.110137,-0.009237,159.476596
2022-12-28,-0.006037,140.3362,-0.005506,193.694488,-0.031908,161.731949,-0.075378,93.650192,0.027183,152.191463,-0.001938,159.167778
2022-12-29,0.039601,146.0052,-0.045115,185.150149,0.019721,164.953133,-0.062374,87.987268,-0.010820,150.553629,0.008130,160.467134
2022-12-30,0.000753,146.1152,-0.033358,179.075763,0.009641,166.551140,0.028566,90.536972,-0.042861,144.237116,0.057755,170.007687


### Descriptive Statistics of the simulated log-return groups

In [10]:
dfStat_mcr = pd.DataFrame()

for symbol, df in fwdDict.items():
    dfPrice_mcr = np.log(df[[f'cl-{n + 1}' for n in range(numSim)]] / df['cl-0'].iloc[0])
    dfFinal_mcr = dfPrice_mcr.iloc[[-1]].transpose()
    dfStat_mcr[symbol] = dfFinal_mcr.describe().iloc[:, 0]
    
dfStat_mcr

NameError: name 'numSim' is not defined

###

### Visualize the simulated equity curves, the actual curve